# Aquecimento global no século 21

Neste notebook, iremos analisar como diferentes nações estão lidando com o maior desafio do século 21: aquecimento global.

conda env create -p (caminho alvo) --file=(caminho do .yml)

In [1]:
# imports padrões
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as pld
import pandas as pd
import seaborn as sns


Bad key "text.kerning_factor" on line 4 in
c:\Users\Leonardo\miniconda3\envs\env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution
